# Using OpenAI functions
This walkthrough demonstrates how to incorporate OpenAI function-calling API's in a chain. We'll go over: 
1. How to use functions to get structured outputs from ChatOpenAI
2. How to create a generic chain that uses (multiple) functions
3. How to create a chain that actually executes the chosen function

In [1]:
from typing import Optional

from langchain.chains.openai_functions import (
    create_openai_fn_chain, create_structured_output_chain
)
from langchain.prompts import ChatPromptTemplate

## Getting structured outputs
We can take advantage of OpenAI functions to try and force the model to return a particular kind of structured output. We'll use the `create_structured_output_chain` to create our chain, which takes the desired structured output either as a Pydantic object or as JsonSchema that conforms to the [OpenAI function spec](https://platform.openai.com/docs/api-reference/chat/create#chat/create-functions).

### Using Pydantic objects
When passing in Pydantic objects to structure our text, we need to make sure to have a docstring description for the class. It also helps to have descriptions for each of the object attributes. Also, keep in mind that the object description should treat the object as a function, so it should describe it as performing some action (even though we don't have an actual function we intend to call).

In [2]:
from pydantic import BaseModel, Field

class Person(BaseModel):
    """Record some identifying information about a person."""
    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")
        
chain = create_structured_output_chain(Person, verbose=True)
chain.run("Sally is 13")



> Entering new  chain...
Prompt after formatting:
Human: Sally is 13

> Finished chain.


Person(name='Sally', age=13, fav_food=None)

To extract arbitrarily many structured outputs of a given format, we can just create a wrapper Pydantic object  that takes a sequence of the original object.

In [3]:
from typing import Sequence

class People(BaseModel):
    """Record identifying information about all people in a text."""
    people: Sequence[Person] = Field(..., description="The people in the text")
        
chain = create_structured_output_chain(People, verbose=True)
chain.run("Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally, so she's 23.")



> Entering new  chain...
Prompt after formatting:
Human: Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally, so she's 23.

> Finished chain.


People(people=[Person(name='Sally', age=13, fav_food=''), Person(name='Joey', age=12, fav_food='spinach'), Person(name='Caroline', age=23, fav_food='')])

### Using JsonSchema

We can also pass in JsonSchema instead of Pydantic objects to specify the desired structure. When we do this, our chain will output json corresponding to the properties described in the JsonSchema, instead of a Pydantic object.

In [13]:
json_schema = {
  "name": "Person",
  "description": "Record some identifying information about a person.",
  "parameters": {
    "title": "Person",
    "description": "Record some identifying information about a person.",
    "type": "object",
    "properties": {
      "name": {
        "title": "Name",
        "description": "The person's name",
        "type": "string"
      },
      "age": {
        "title": "Age",
        "description": "The person's age",
        "type": "integer"
      },
      "fav_food": {
        "title": "Fav Food",
        "description": "The person's favorite food",
        "type": "string"
      }
    },
    "required": [
      "name",
      "age"
    ]
  }
}

In [17]:
chain = create_structured_output_chain(json_schema, verbose=True)
chain.run("Sally is 13")



> Entering new  chain...
Prompt after formatting:
Human: Sally is 13

> Finished chain.


{'name': 'Sally', 'age': 13}

## Creating generic OpenAI functions chain
To create a chain that takes in multiple functions, we can use the `create_openai_fn_chain` method, which acts the same as `create_structured_output_chain` except that it can take multiple functions **and** it can take pure Python functions as arguments.

### Using Python functions
We can pass in functions as Pydantic objects, JsonSchema, or Python functions. To pass Python function in directly, we'll want to make sure our parameters have type hints, we have a docstring, and we use [Google Python style docstrings](https://google.github.io/styleguide/pyguide.html#doc-function-args) to describe the parameters.

**NOTE**: To use Python functions, make sure the function arguments are of primitive types (str, float, int, bool) or that they are Pydantic objects.

In [24]:
class OptionalFavFood(BaseModel):
    food: Optional[str] = None

def record_person(name: str, age: int, fav_food: OptionalFavFood) -> str:
    """Record some basic identifying information about a person.
    
    Args:
        name: The person's name.
        age: The person's age in years.
        fav_food: The name of the person's favorite food.
    """
    return f"Recording person {name} of age {age} with favorite food {fav_food.food}!"

    
chain = create_openai_fn_chain([record_person], verbose=True)
chain.run("Tommy was 12, and loved apple pie")



> Entering new  chain...
Prompt after formatting:
Human: Tommy was 12, and loved apple pie

> Finished chain.


{'name': 'Tommy', 'age': 12, 'fav_food': {'food': 'apple pie'}}

If we pass in multiple Python functions or JsonSchemas, then the returned output will be of the form
```python
{"name": "<<function_name>>", "arguments": "<<function_arguments>>"}
```

In [10]:
def record_dog(name: str, color: str, fav_food: Optional[str]=None) -> str:
    """Record some basic identifying information about a dog.
    
    Args:
        name: The dog's name.
        color: The dog's color.
        fav_food: The name of the dog's favorite food.
    """
    return f"Recording dog {name} of color {color} with favorite food {fav_food}!"


chain = create_openai_fn_chain([record_person, record_dog], verbose=True)
chain.run("Harry was a chubby brown beagle who loved chicken")



> Entering new  chain...
Prompt after formatting:
Human: Harry was a chubby brown beagle who loved chicken

> Finished chain.


{'name': 'record_dog',
 'arguments': {'name': 'Harry', 'color': 'brown', 'fav_food': 'chicken'}}

If we pass in multiple Pydantic objects, the returned object will be identifiable by it's class so we don't need to return the name explicitly.

In [11]:
class Dog(BaseModel):
    """Record some identifying information about a dog."""
    name: str = Field(..., description="The dog's name")
    color: str = Field(..., description="The dog's color")
    fav_food: Optional[str] = Field(None, description="The dog's favorite food")

chain = create_openai_fn_chain([Person, Dog], verbose=True)
chain.run("Harry was a chubby brown beagle who loved chicken")



> Entering new  chain...
Prompt after formatting:
Human: Harry was a chubby brown beagle who loved chicken

> Finished chain.


Dog(name='Harry', color='brown', fav_food='chicken')

## Create a Chain that runs the chosen function
We can go one step further and create a chain that actually executes the function chosen by the model.

In [30]:
import inspect
from typing import Any, Callable, Dict, List, Optional

from langchain.callbacks.manager import CallbackManagerForChainRun
from langchain.chains.base import Chain
from langchain.input import get_colored_text


class FunctionExecutorChain(Chain):
    functions: Dict[str, Callable]
    output_key: str = "output"
    input_key: str = "function"

    @property
    def input_keys(self) -> List[str]:
        return [self.input_key]

    @property
    def output_keys(self) -> List[str]:
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        """Run the logic of this chain and return the output."""
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        name = inputs["function"].pop("name")
        args = inputs["function"].pop("arguments")
        _pretty_name = get_colored_text(name, "green")
        _pretty_args = get_colored_text(json.dumps(args, indent=2), "green")
        _text = f"Calling function {_pretty_name} with arguments:\n" + _pretty_args
        _run_manager.on_text(_text)
        _args = {}
        function = self.functions[name]
        for arg_name, arg_type in inspect.get_annotations(function).items():
            if isinstance(arg_type, type) and issubclass(arg_type, BaseModel):
                args[arg_name] = arg_type.parse_obj(args[arg_name])
        output = function(**args)
        return {self.output_key: output}

In [31]:
from langchain.chains import SequentialChain
from langchain.chains.openai_functions.base import convert_to_openai_function

functions = [record_person, record_dog]
openai_functions = [convert_to_openai_function(f) for f in functions]
fn_map = {
    openai_fn["name"]: fn for openai_fn, fn in zip(openai_functions, functions)
}
llm_chain = create_openai_fn_chain(functions)
exec_chain = FunctionExecutorChain(functions=fn_map, verbose=True)
chain = SequentialChain(
    chains=[llm_chain, exec_chain],
    input_variables=llm_chain.input_keys,
    output_variables=["output"],
    verbose=True
)
chain.run("Tommy was 12, and loved apple pie")



> Entering new  chain...


> Entering new  chain...
Calling function record_person with arguments:
{
  "name": "Tommy",
  "age": 12,
  "fav_food": {
    "food": "apple pie"
  }
}
> Finished chain.

> Finished chain.


'Recording person Tommy of age 12 with favorite food apple pie!'